In [ ]:
import progressbar as pgb

pgb.streams.wrap_stderr()

import functools
import logging

import atlite
import geopandas as gpd
import hvplot
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import colors
from rasterio.plot import show

logger = logging.getLogger(__name__)

from _helpers import configure_logging

if __name__ == "__main__":

    configure_logging(snakemake)

    technology_details = snakemake.params["technology_details"]

    cutout = atlite.Cutout(snakemake.input["cutout"])

    # Full region recognised
    region = gpd.read_file(snakemake.input["region"]).set_index("index")
    # Region in which RES technology can be build
    build_region = region.loc[technology_details["build_in"]]

    # Initialise excluder with Mollweide CRS
    # Could be made more accurate by choosing a more accurate (local) CRS
    crs_ea = "ESRI:54009"

    # Same CRS for region and exclusion calculation required
    region = region.to_crs(crs_ea)
    build_region = build_region.to_crs(crs_ea)
    excluder = atlite.ExclusionContainer(crs=crs_ea, res=100)

    if "wdpa" in technology_details:
        # WDPA layers "points" and "polygons" are treated separately

        # Polygons are excluded with their presented shape
        if "polygons" in technology_details["wdpa"]["layers"]:
            wdpa = gpd.read_file(
                snakemake.input["wdpa"],
                # Only load geometries intersecting the cutout
                bbox=tuple(cutout.bounds),
                # polygon layer, points layer optionally later
                layer="WDPA_Jan2022_Public_shp-polygons",
            )

            # Limit considered protected areas to configured statuses to include
            wdpa[wdpa["STATUS"].isin(technology_details["wdpa"]["include_status"])]

            # Ensure correct CRS
            wdpa = wdpa.to_crs(crs_ea)

            logging.info(f"Excluding {len(wdpa)} PA polygon areas.")
            if not wdpa.empty:
                excluder.add_geometry(wdpa["geometry"])

        # Points are assumed to be the center of an protected area
        # A circular area around this assumed center with the reported
        # area "REP_AREA" is excluded for points to estimate the protected area
        if "points" in technology_details["wdpa"]["layers"]:
            wdpa = gpd.read_file(
                snakemake.input["wdpa"],
                # Only load geometries intersecting the cutout
                # for the points layer treated here this
                # excludes points outside the bounds for which the
                # modelled area would overlap into the cutout.bounds
                # methodological weakness: accepted here
                bbox=tuple(cutout.bounds),
                layer="WDPA_Jan2022_Public_shp-points",
            )

            # Limit considered protected areas to configured statuses to include
            wdpa[wdpa["STATUS"].isin(technology_details["wdpa"]["include_status"])]

            # Ensure correct CRS
            wdpa = wdpa.to_crs(crs_ea)

            # Only model areas for points with relevant reported areas (> 1 km)
            wdpa = wdpa[wdpa["REP_AREA"] > 1]

            # Calculate radius in km to buffer around central point
            wdpa["buffer_radius"] = np.sqrt(wdpa["REP_AREA"] / np.pi)

            wdpa = wdpa.set_geometry(wdpa["geometry"].buffer(wdpa["buffer_radius"]))

            logging.info(f"Excluding {len(wdpa)} PA point areas.")
            if not wdpa.empty:
                excluder.add_geometry(wdpa["geometry"])

    if "wdpa_marine" in technology_details:
        # WDPA layers "points" and "polygons" are treated separately

        # Polygons are excluded with their presented shape
        if "polygons" in technology_details["wdpa_marine"]["layers"]:
            wdpa_marine = gpd.read_file(
                snakemake.input["wdpa_marine"],
                # Only load geometries intersecting the cutout
                bbox=tuple(cutout.bounds),
                # polygon layer, points layer optionally later
                layer="WDPA_WDOECM_Jan2022_Public_marine_shp-polygons",
            )

            # Limit considered protected areas to configured statuses to include
            wdpa_marine[
                wdpa_marine["STATUS"].isin(
                    technology_details["wdpa_marine"]["include_status"]
                )
            ]

            # Ensure correct CRS
            wdpa_marine = wdpa_marine.to_crs(crs_ea)

            logging.info(f"Excluding {len(wdpa_marine)} MPA polygon areas.")
            if not wdpa_marine.empty:
                excluder.add_geometry(wdpa_marine["geometry"])

        # Points are assumed to be the center of an protected area
        # A circular area around this assumed center with the reported
        # area "REP_AREA" is excluded for points to estimate the protected area
        if "points" in technology_details["wdpa_marine"]["layers"]:
            wdpa_marine = gpd.read_file(
                snakemake.input["wdpa_marine"],
                # Only load geometries intersecting the cutout
                # for the points layer treated here this
                # excludes points outside the bounds for which the
                # modelled area would overlap into the cutout.bounds
                # methodological weakness: accepted here
                bbox=tuple(cutout.bounds),
                layer="WDPA_WDOECM_Jan2022_Public_marine_shp-points",
            )

            # Limit considered protected areas to configured statuses to include
            wdpa_marine[
                wdpa_marine["STATUS"].isin(
                    technology_details["wdpa_marine"]["include_status"]
                )
            ]

            # Ensure correct CRS
            wdpa_marine = wdpa_marine.to_crs(crs_ea)

            # Only model areas for points with relevant reported areas (> 1 km)
            wdpa_marine = wdpa_marine[wdpa_marine["REP_AREA"] > 1]

            # Calculate radius in km to buffer around central point
            wdpa_marine["buffer_radius"] = np.sqrt(wdpa_marine["REP_AREA"] / np.pi)

            wdpa_marine = wdpa_marine.set_geometry(
                wdpa_marine["geometry"].buffer(wdpa_marine["buffer_radius"])
            )

            logging.info(f"Excluding {len(wdpa_marine)} MPA point areas.")
            if not wdpa_marine.empty:
                excluder.add_geometry(wdpa_marine["geometry"])

    if "gebco" in technology_details:
        # lambda not supported for atlite + multiprocessing
        # use named function np.greater with partially frozen argument instead
        # and exclude areas where: -max_depth > grid cell depth
        func = functools.partial(np.greater, -technology_details["gebco"]["max_depth"])
        excluder.add_raster(
            snakemake.input["gebco"], codes=func, crs="EPSG:4326", nodata=-1000
        )

    if "copernicus" in technology_details:
        # Eligible land use/cover codes for building RES
        excluder.add_raster(
            snakemake.input["copernicus"],
            crs="EPSG:4236",
            codes=technology_details["copernicus"]["include_codes"],
            invert=True,
        )

        # Buffer distance around these land use/cover codes blocked
        # for RES build-up
        if technology_details["copernicus"]["distancing_codes"]:
            excluder.add_raster(
                snakemake.input["copernicus"],
                crs="EPSG:4236",
                codes=technology_details["copernicus"]["distancing_codes"],
                buffer=technology_details["copernicus"]["distance_to_codes"],
            )

    if "max_shore_distance" in technology_details:
        # Include only offshore/eez areas within the configured shore distance
        excluder.add_geometry(
            region.loc[["onshore"]],
            buffer=technology_details["max_shore_distance"],
            invert=True,
        )

    # Determine eligible/excluded areas on exclude raster (100m)
    masked, transform = atlite.gis.shape_availability(
        build_region["geometry"], excluder
    )
    eligible_share = (
        masked.sum() * excluder.res ** 2 / build_region.geometry.unary_union.area
    )

    ## Plot mask for technology of all eligible areas in the relevant parts of the region
    masked[
        masked == 0
    ] = np.nan  # Otherwise plotting masks is not transparent for plot layers below

    # create colormap with single color (= Green) to plot the mask
    cmap = colors.ListedColormap(["green"])
    fig, ax = plt.subplots(figsize=(8, 6))
    # Region to not build in
    region[~region.index.isin(technology_details["build_in"])].plot(
        ax=ax, edgecolor="red", color="None", hatch="///", zorder=1
    )
    # Region to build in
    build_region.plot(ax=ax, edgecolor="k", fc="lightgray", alpha=0.5, lw=1, zorder=5)
    # Mask for eligible region
    show(
        masked,
        transform=transform,
        interpolation="nearest",
        cmap=cmap,
        ax=ax,
        zorder=20,
    )
    ax.set_title(
        f'{snakemake.wildcards["region"]}: Eligible area (green, {eligible_share * 100:2.2f}%)'
    )
    ax.set_xlabel("Longitude [km]")
    ax.set_ylabel("Latitude [km]")
    fig.savefig(snakemake.output["area_mask"])

    ## Determine eligible/excluded areas on cutout raster (grid cell level)
    logger.info("Calculating eligible RES build areas...")
    availability = cutout.availabilitymatrix(build_region, excluder)
    availability = availability.rename("Grid cell share available for RES")
    availability.attrs["units"] = "p.u."

    # Combine all indices from build_region into one availability
    availability = availability.sum(dim="index")

    # DataArray representing the area of each cutout grid cell
    area = cutout.grid.to_crs(3035).area / 1e6
    area = xr.DataArray(
        area.values.reshape(cutout.shape), [cutout.coords["y"], cutout.coords["x"]]
    )
    area.attrs["units"] = "km^2"

    potential = technology_details["technical_potential"] * availability * area
    potential = potential.rename("Technical potential per grid cell")
    potential.attrs["units"] = "MW"

    ## Calculate each grid cells capacity factor to create quality classes
    func = getattr(cutout, technology_details["atlite"].pop("method"))
    capacity_factor = func(capacity_factor=True, **technology_details["atlite"])

    ## Plot/save technical potential (map) and capacity factor (map)
    plot = potential.hvplot(
        title=f"{potential.name} [{potential.attrs['units']}]", cmap="viridis"
    ) * build_region.to_crs(cutout.crs).dissolve().hvplot(color="None")
    hvplot.save(plot, snakemake.output["potential_map"])

    plot = capacity_factor.drop(["lat", "lon"]).hvplot(
        x="x",
        y="y",
        title="Annual capacity factor per grid cell [p.u.]",
        cmap="inferno",
    ) * build_region.to_crs(cutout.crs).dissolve().hvplot(color="None")
    hvplot.save(plot, snakemake.output["capacity_factor_map"])

    ## Create masks for capacity layouts by binning into "number_quality_classes"
    cf_min, cf_max, nbins = (
        capacity_factor.min(),
        capacity_factor.max(),
        technology_details["number_quality_classes"],
    )
    bins = np.linspace(cf_min, cf_max, nbins + 1)

    masks = []
    for i in range(nbins):
        m = np.logical_and(bins[i] <= capacity_factor, capacity_factor < bins[i + 1])

        masks.append(m)

    idx = np.round(bins[:-1] + np.diff(bins), 3)
    masks = xr.concat(masks, pd.Index(idx, name="capacity factor quality class"))

    ## Calculate capacities and profiles for each quality class
    profile, capacities = func(
        matrix=masks.stack(spatial=["y", "x"]),
        layout=potential,
        index=masks.coords["capacity factor quality class"],
        per_unit=True,
        return_capacity=True,
        **technology_details["atlite"],
    )

    ## Merge and save
    ds = xr.merge([profile.rename("profiles"), capacities.rename("capacities")])
    ds.to_netcdf(snakemake.output["profiles"])